In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, sin, cos, sqrt, atan2

df = pd.read_csv("dataset.csv")

df['LastUpdatedDate'] = df['LastUpdatedDate'].astype(str)
df['LastUpdatedTime'] = df['LastUpdatedTime'].astype(str)
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'])

numeric_cols = ['Capacity', 'Occupancy', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['VehicleType'] = df['VehicleType'].astype(str).str.lower()

def baseline_price_model(df, alpha=2.0):
    df = df.sort_values(by=['SystemCodeNumber', 'timestamp']).copy()
    df['price'] = 10.0
    for i in range(1, len(df)):
        if df.iloc[i]['SystemCodeNumber'] == df.iloc[i - 1]['SystemCodeNumber']:
            occ_ratio = df.iloc[i]['Occupancy'] / df.iloc[i]['Capacity']
            df.loc[df.index[i], 'price'] = min(max(df.iloc[i - 1]['price'] + alpha * occ_ratio, 5), 20)
    return df

def demand_price_model(df, base_price=10, alpha=1.5, beta=1.0, gamma=1.0, delta=2.0, epsilon=2.0, lam=0.5):
    df = df.copy()
    df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
    df['norm_queue'] = df['QueueLength'] / df['QueueLength'].max()
    df['norm_traffic'] = df['TrafficConditionNearby'] / df['TrafficConditionNearby'].max()
    df['vehicle_weight'] = df['VehicleType'].map({'car': 1.0, 'bike': 0.5, 'truck': 1.5}).fillna(1.0)

    df['raw_demand'] = (alpha * df['occupancy_rate'] +
                        beta * df['norm_queue'] -
                        gamma * df['norm_traffic'] +
                        delta * df['IsSpecialDay'] +
                        epsilon * df['vehicle_weight'])

    min_d, max_d = df['raw_demand'].min(), df['raw_demand'].max()
    df['norm_demand'] = 2 * ((df['raw_demand'] - min_d) / (max_d - min_d)) - 1

    df['price'] = base_price * (1 + lam * df['norm_demand'])
    df['price'] = df['price'].clip(lower=5, upper=20)
    return df

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def competitive_price_model(df, base_price=10, alpha=1.5, beta=1.0, gamma=1.0, delta=2.0, epsilon=2.0, lam=0.5):
    df = demand_price_model(df, base_price, alpha, beta, gamma, delta, epsilon, lam)
    df = df.copy()
    df['final_price'] = df['price']

    grouped = df.groupby('SystemCodeNumber')
    for lot_id, lot_df in grouped:
        lat1 = lot_df.iloc[0]['Latitude']
        lon1 = lot_df.iloc[0]['Longitude']
        curr_price = lot_df['price'].mean()

        others = df[df['SystemCodeNumber'] != lot_id].copy()
        others.loc[:, 'distance'] = others.apply(lambda x: haversine(lat1, lon1, x['Latitude'], x['Longitude']), axis=1)
        nearby = others[others['distance'] <= 1.0]

        if not nearby.empty:
            avg_price = nearby['price'].mean()
            if avg_price < curr_price:
                adjusted_price = curr_price * 0.95
            elif avg_price > curr_price:
                adjusted_price = curr_price * 1.05
            else:
                adjusted_price = curr_price
        else:
            adjusted_price = curr_price

        df.loc[df['SystemCodeNumber'] == lot_id, 'final_price'] = min(max(adjusted_price, 5), 20)

    df['price'] = df['final_price']
    return df

df_baseline = baseline_price_model(df)
df_demand = demand_price_model(df)
df_model3 = competitive_price_model(df)

import matplotlib.pyplot as plt
lot_ids = df['SystemCodeNumber'].unique()
lot = lot_ids[0]

plt.figure(figsize=(12, 6))
plt.plot(df_baseline[df_baseline['SystemCodeNumber'] == lot]['timestamp'],
         df_baseline[df_baseline['SystemCodeNumber'] == lot]['price'], label='Model 1')
plt.plot(df_demand[df_demand['SystemCodeNumber'] == lot]['timestamp'],
         df_demand[df_demand['SystemCodeNumber'] == lot]['price'], label='Model 2')
plt.plot(df_model3[df_model3['SystemCodeNumber'] == lot]['timestamp'],
         df_model3[df_model3['SystemCodeNumber'] == lot]['price'], label='Model 3')

plt.title(f"Price Trends for Lot {lot}")
plt.xlabel("Timestamp")
plt.ylabel("Price ($)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
